In [ ]:
%matplotlib inline

from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

from numpy import exp, log, sqrt
from scipy.linalg import expm, inv, solve_banded

In [ ]:
x_start = np.linspace(-0.99, 0.99, 20)

d = 1
T = 10
delta_t = 0.001
sq_delta_t = sqrt(delta_t)
N = int(np.ceil(T  / delta_t))
K = 10000

exp_hitting_times = []
exp_hitting_times_pos = []

for x in x_start:
    
    hitting_times = np.zeros(K)

    X = x * np.ones([K, d])

    for n in range(N):
        selection = np.sqrt(np.sum(X**2, 1)) < 1 
        K_selection = np.sum(selection)
        if K_selection == 0:
            break
        xi = np.random.randn(K_selection, d)
        X[selection, :] = X[selection, :] + sqrt(2) * sq_delta_t * xi
        hitting_times[selection] += delta_t
        
    exp_hitting_times.append(np.mean(exp(-hitting_times)))
    exp_hitting_times_pos.append(np.mean(exp(hitting_times)))


In [ ]:
x_val = np.linspace(-1, 1, 100)
plt.plot(x_val, (exp(-x_val + 1) + exp(x_val + 1)) / (1 + exp(2)));
plt.plot(x_start, exp_hitting_times)
plt.plot(x_val, np.cos(x_val) / np.cos(1));
plt.plot(x_start, exp_hitting_times_pos);

In [ ]:
def u(x, t):
    return np.sqrt(2) * (-exp(-2 * x) + 1) / (exp(-2 * x) + 1)


def do_importance_sampling(epsilon, K, delta_t, verbose=True):

    d = 1
    T = 10
    
    B = np.sqrt(2) * np.eye(d)

    sq_delta_t = np.sqrt(delta_t)
    N = int(T / delta_t)

    X = np.zeros([d, K])
    X_u = np.zeros([d, K])
    ito_int = np.zeros(K)
    riemann_int = np.zeros(K)
    f_int = np.zeros(K)
    f_int_u = np.zeros(K)
    hitting_times = np.zeros(K)
    hitting_times_u = np.zeros(K)

    for n in range(N + 1):
        selection = (X[0, :] > -1) & (X[0, :] < 1)
        K_ = np.sum(selection)
        if K_ == 0:
            break
        xi = np.random.randn(d, K_)
        X[:, selection] = X[:, selection] + B.dot(xi) * sq_delta_t
        hitting_times[selection] += delta_t

    print(n)

    for n in range(N + 1):
        selection = (X_u[0, :] > -1) & (X_u[0, :] < 1)
        K_ = np.sum(selection)
        if K_ == 0:
            break
        xi = np.random.randn(d, K_)
        ut = u(X_u[:, selection], n * delta_t) + epsilon
        X_u[:, selection] = X_u[:, selection] + B.dot(ut) * delta_t + B.dot(xi) * sq_delta_t
        ito_int[selection] += np.sum(ut * xi, 0) * sq_delta_t
        riemann_int[selection] += np.sum(ut**2, 0) * delta_t
        hitting_times_u[selection] += delta_t

    print(n)

    girsanov = np.exp(- ito_int - 0.5 * riemann_int)

    stats = {}
    stats['naive'] = {}
    stats['IS'] = {}
    stats['naive']['mean'] = np.mean(exp(-hitting_times))
    stats['naive']['variance'] = np.var(exp(-hitting_times))
    stats['naive']['relative_error'] = np.sqrt(stats['naive']['variance']) / stats['naive']['mean']
    stats['IS']['mean'] = np.mean(exp(-hitting_times_u) * girsanov)
    stats['IS']['variance'] = np.var(exp(-hitting_times_u) * girsanov)
    stats['IS']['relative_error'] = np.sqrt(stats['IS']['variance']) / stats['IS']['mean']

    if verbose == True:
        print('naive mean: %.4e, naive variance: %.4e, naive RE: %.4e' % (stats['naive']['mean'], stats['naive']['variance'], stats['naive']['relative_error']))
        print('IS mean:    %.4e, IS variance:    %.4e, IS RE:    %.4e' % (stats['IS']['mean'], stats['IS']['variance'], stats['IS']['relative_error']))
        #print('true mean:  %.4e' % np.exp(-v(np.zeros(d), 0)))
        
    return stats, np.mean(hitting_times)

In [ ]:
epsilon_range = np.linspace(0, 2.0, 10)

B = np.sqrt(2) * np.eye(d)

# simulate hitting times 2u* - u

K = 100000
delta_t = 0.0001
sq_delta_t = np.sqrt(delta_t)
N = int(T / delta_t)
    
hitting_times_u_eps = []
RE_ = []

for epsilon in epsilon_range:
    X_u = np.zeros([d, K])

    hitting_times_2u_u = np.zeros(K)

    for n in range(N + 1):
        selection = (X_u[0, :] > -1) & (X_u[0, :] < 1)
        K_ = np.sum(selection)
        if K_ == 0:
            break
        xi = np.random.randn(d, K_)
        ut = u(X_u[:, selection], n * delta_t) - epsilon
        X_u[:, selection] = X_u[:, selection] + B.dot(ut) * delta_t + B.dot(xi) * sq_delta_t
        hitting_times_2u_u[selection] += delta_t
        
    hitting_times_u_eps.append(np.mean(hitting_times_2u_u))
    RE_.append(np.sqrt(np.mean(np.exp(d * epsilon**2 * hitting_times_2u_u)) - 1))

In [ ]:
RE = []
hitting_times = []

epsilon_range = np.linspace(0, 2.0, 10)

for epsilon in epsilon_range:
 
    stats, mean_hitting_times = do_importance_sampling(K=100000, delta_t=0.0001, epsilon=epsilon, verbose=False)
    RE.append(stats['IS']['relative_error'])
    hitting_times.append(mean_hitting_times)

In [ ]:
plt.plot(epsilon_range, RE)
plt.plot(epsilon_range, RE_)
plt.plot(epsilon_range, np.sqrt(np.exp(d * epsilon_range**2 * np.array(hitting_times)) - 1));
plt.plot(epsilon_range, np.sqrt(np.exp(d * epsilon_range**2 * np.array(hitting_times_u_eps)) - 1));

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3.5))
ax.plot(epsilon_range, RE, label='sampled');
ax.plot(epsilon_range, RE_, '--', label='correct formula')
ax.plot(epsilon_range, np.sqrt(np.exp(d * epsilon_range**2 * np.array(hitting_times)) - 1), '--', label=r'formula with $T \leftrightarrow \mathbb{E}[\tau]$');
ax.plot(epsilon_range, np.sqrt(np.exp(d * epsilon_range**2 * np.array(hitting_times_u_eps)) - 1), '--', label=r'formula with $T \leftrightarrow \mathbb{E}[\tau^{2u^* - u}]$');
ax.set_xlabel(r'$\varepsilon$')
ax.set_ylabel(r'$r(u)$')
ax.set_title(r'Brownian motion with hitting time')
ax.legend()

#fig.savefig('img/suboptimal_control_brownian_motion_hitting.pdf')